In [1]:
import pandas as pd
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import os
import glob

In [2]:
# redorder the list of tracked points in proboscis 

def reorder_columns(string, df):
    list_of_coord = list([column for column in df.columns if string in column and 'prob' in column])
    myorder = [0, 2, 3, 4, 5, 6, 1]
    mylist = [list_of_coord[i] for i in myorder]
    return(mylist)

In [6]:
def fitAThirdDegCurve(xs, ys):
    p = np.poly1d(np.polyfit(ys, xs, deg = 3))
    return(p)

In [3]:
filepath = './dataFolder/TrackingData_2022_11_10/thresholded_bodyparts/'
files = glob.glob(filepath + '*.csv')

In [18]:
# trackPoints = pd.read_csv('./dataFolder/MV_m1_points.csv')
for f in files:
    trackPoints = pd.read_csv(f)
    mothID = f.split('\\')[-1][:-14]
    
    trackPoints.drop(trackPoints.columns[0], axis=1, inplace=True)
    arr = []
    for coord in ['_x', '_y']: #NOTE: for manually digitized files in DLTdv the variables will be X and Y, not x and y. 
        name = reorder_columns(coord, trackPoints)
        arr.append(name)
    X_arr = trackPoints.loc[:, arr[0]]
    Y_arr = trackPoints.loc[:, arr[1]]
    
    X_arr.to_csv('./dataFolder/TrackingData_2022_11_10/x_y_ordered_polyFit/' + mothID + '_x_proboscis_ordered.csv')
    Y_arr.to_csv('./dataFolder/TrackingData_2022_11_10/x_y_ordered_polyFit/' + mothID + '_y_proboscis_ordered.csv')
    
    # fit a polyfit of nth degree
    p_fit = []
    for rr, x in X_arr.iterrows():
            y = Y_arr.loc[rr,:].dropna()
            x = x.dropna()
            p = fitAThirdDegCurve(x.values, y.values)
            p_fit.append(p)
    np.save('./dataFolder/TrackingData_2022_11_10/x_y_ordered_polyFit/' + mothID + '_3DpolyfitVariables.npy', p_fit)

C:\Users\tanvi.deora\AppData\Local\Temp\ipykernel_27776\289051980.py:22: RankWarning: Polyfit may be poorly conditioned
  p = fitAThirdDegCurve(x.values, y.values)


## run the following cells to make figure of the proboscis fit line - see code 2a instead

In [ ]:
def MakeFigureForFit(x, y, p, ax):
    ax.plot(x, y, '.b', p(y),y,'-k')
    ax.set_xlim([x.min()-10,x.max() + 10])
    ax.set_ylim([y.min() -10, y.max() + 10])
    return()

In [ ]:
folder = './dataFolder/computeInterpolation/Polyfit3/'

In [ ]:
# make figures to access the interpolation

f = plt.figure()
ax = f.add_axes([0.3, 0.3, 0.7, 0.7])

for ii, p in enumerate(p_fit):
    x = X_arr.loc[ii,:].values
    y = Y_arr.loc[ii,:].values
    MakeFigureForFit(x,y,p,ax)
    figname = str(ii).zfill(4) + '.png'
    plt.savefig(folder + figname)

    plt.cla()

In [ ]:
# from scipy import interpolate
# 
# def interpol_points(x, y):
#     tck, u = interpolate.splprep([x, y], s=3)
#     new_points = interpolate.splev(u, tck, der=0)
#     return(new_points)

In [ ]:
# new_x = []
# new_y = []
# for r, data in X_arr.iterrows():
#     x = data.values
#     y = Y_arr.loc[r, :].values
#     pts__ = interpol_points(x, y)
#     x__ = pts__[0]
#     y__ = pts__[1]
#     new_x.append(x__)
#     new_y.append(y__)